# Radar Data Smoothing with DCT: Demo

This notebook demonstrates the **Discrete Cosine Transform (DCT)** based smoothing algorithm for radar data in polar coordinates. 

The method is described in the paper: *"Radar Data Smoothing using the Discrete Cosine Transform: A Fast Spectral Domain Algorithm"*.

**Key Advantage:** It correctly handles the varying physical width of azimuth beams (the "pie slice" effect) by adapting the smoothing kernel in the spectral domain, achieving $O(N \log N)$ complexity.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import dct_smoothing  # The module we created

# Set random seed for reproducibility
np.random.seed(42)

## 1. Generate Synthetic Radar Data

We generate a smooth synthetic field in Cartesian coordinates and sample it onto a polar radar grid. This avoids singularity artifacts at the origin and provides a known "Ground Truth".

In [ ]:
# Define Radar Grid Parameters
n_az = 360      # Number of azimuth rays
n_range = 1000  # Number of range gates
az_res_deg = 1.0
range_res_m = 50.0

azimuths = np.linspace(0, 360, n_az, endpoint=False)
ranges = np.arange(n_range) * range_res_m

# Create Meshgrid for Polar Coordinates
AZ_deg, RG_m = np.meshgrid(azimuths, ranges, indexing='ij')
AZ_rad = np.deg2rad(AZ_deg)

# Convert to Cartesian (x, y) for synthetic pattern generation
X = RG_m * np.sin(AZ_rad) / 1000.0  # km (East)
Y = RG_m * np.cos(AZ_rad) / 1000.0  # km (North)

# Generate Ground Truth Pattern
# A mix of waves and a Gaussian blob
ground_truth = (
    25.0 +  
    8.0 * np.sin(2 * np.pi * X / 40.0) +           # Wave 1
    6.0 * np.cos(2 * np.pi * Y / 30.0) +           # Wave 2
    4.0 * np.exp(-((X - 20)**2 + (Y - 15)**2) / 200.0) # Gaussian feature
)

# Add Noise
noise_std = 3.0
noise = np.random.normal(0, noise_std, size=ground_truth.shape)
noisy_data = ground_truth + noise

# Add some NaNs to demonstrate handling
noisy_data[100:110, 200:250] = np.nan

print(f"Data Shape: {noisy_data.shape}")
print(f"Range: {ranges[0]}m to {ranges[-1]}m")

## 2. Apply DCT Smoothing

We apply the DCT smoothing with a target physical width. The algorithm automatically adjusts the effective azimuth window size based on range.

In [ ]:
smoothing_width = 15.0  # pixels (scaling factor for physical width)
# Ideally this maps to physical width = width * range_res or similar. 
# In the module logic, 'width_pixels' maintains a consistent arc length relative to range bins.

print("Applying DCT Smoothing...")
smoothed_dct = dct_smoothing.apply_polar_dct_smoothing(
    noisy_data, 
    az_res_deg=az_res_deg, 
    width_pixels=smoothing_width, 
    kernel_type='boxcar'
)
print("Done.")

## 3. Visualize Results

We compare the original Ground Truth, the Noisy Input, and the Smoothed Output using a Polar Plot (PPI).

In [ ]:
def plot_ppi(data, title, ax, vmin=10, vmax=40):
    # Simple Cartesian pcolormesh for visualization
    # (In a real radar lib like Py-ART this would be more complex, but this suffices for demo)
    mesh = ax.pcolormesh(X, Y, data, cmap='pyart_HomeyerRainbow', vmin=vmin, vmax=vmax, shading='auto')
    ax.set_title(title)
    ax.set_xlabel("East (km)")
    ax.set_ylabel("North (km)")
    ax.set_aspect('equal')
    return mesh

# Setup Colormap (fallback if pyart not installed)
try:
    import cmweather
except ImportError:
    # Fallback
    pass
    
fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharex=True, sharey=True)

plot_ppi(ground_truth, "Ground Truth", axes[0])
plot_ppi(noisy_data, "Noisy Input", axes[1])
mesh = plot_ppi(smoothed_dct, "DCT Smoothed (Boxcar)", axes[2])

plt.colorbar(mesh, ax=axes, label="Reflectivity (dBZ)")
plt.suptitle(f"DCT Smoothing Demo (Width={smoothing_width} px)", fontsize=16)
plt.show()

## 4. Error Analysis

Quantify the improvement in Root Mean Square Error (RMSE).

In [ ]:
mask = np.isfinite(noisy_data) & np.isfinite(smoothed_dct)

rmse_noisy = np.sqrt(np.mean((noisy_data[mask] - ground_truth[mask])**2))
rmse_dct = np.sqrt(np.mean((smoothed_dct[mask] - ground_truth[mask])**2))

print(f"RMSE Noisy:  {rmse_noisy:.4f}")
print(f"RMSE DCT:    {rmse_dct:.4f}")
print(f"Improvement: {(rmse_noisy - rmse_dct) / rmse_noisy * 100:.2f}%")